In [31]:
import pandas as pd
import numpy as np
#Import the literal_eval function from ast
from ast import literal_eval

In [22]:
df = pd.read_json('../employers.json')
df.shape[0]

20000

In [23]:
df = df.drop('_id', axis=1)
df.head()

,template,type,shift,city,zip,salary1,salary2,benefits,language,vertrag,futher,startDate,views
0,Logopäde,freelancer,3,Ottendorf-Okrilla,1458,2300,4500,"[Bezahlung nach Tarif(-vertrag), Leistungspräm...",5,1,[Ausbildungsbetrieb],0,1420
1,Podologe,teilzeit,0,Dresden,01067-01465,2100,4600,"[Betriebliches Gesundheitsmanagement, Dienstbe...",0,2,[],0,1423
2,Ergotherapeut,teilzeit,2,Ottendorf-Okrilla,1458,2400,2600,"[Dienstbekleidung, gute Betriebsklima, Übernah...",5,1,[],1,783
3,Krankenpflegehelfer,vollzeit,0,Radeburg,1471,2100,3300,"[gute Betriebsklima, exklusive Rabatte in namh...",0,0,[],0,502
4,Logopäde,teilzeit,3,Moritzburg,1468,2100,2700,"[gute Betriebsklima, Begrüßungsprämie, Vorzugs...",3,1,[],1,586


In [24]:
#Print test
df.iloc[0]['city']

'Ottendorf-Okrilla'

In [25]:
#Define a stringified list and output its type
a = "[1,2,3]"
print(type(a))
#Apply literal_eval and output type
b = literal_eval(a)
print(type(b))

<class 'str'>
<class 'list'>


In [26]:
#Convert all NaN into stringified empty lists
# df['benefits'] = df['benefits'].fillna('[]')

#Apply literal_eval to convert to the list object
# df['benefits'] = df['benefits'].apply(literal_eval)

#Convert list of dictionaries to a list of strings
# df['benefits'] = df['benefits'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
# df.head(5)

In [27]:
#Create a new feature by exploding genres
# s = df.apply(lambda x: pd.Series(x['benefits']),axis=1).stack().reset_index(level=1, drop=True)

#Name the new feature as 'genre'
# s.name = 'benefit'

#Create a new dataframe gen_df which by dropping the old 'benefits' feature and adding the new 'benefit'.
# ben_df = df.drop('benefits', axis=1).join(s)

#Print the head of the new gen_df
# ben_df.head()
ben_df = df.copy()

In [28]:
def build_chart(ben_df, percentile=0.8):
    #Ask for template
    print("Input preferred template")
    template = input()

    #Ask for working type
    print("Working type:")
    wtype = input()

    #Ask for working shift
    print("Working shift")
    wshift = int(input())

    #Ask for city
    print("Where you want to work")
    city = input()

    #Ask for salary range
    print("Salary lowest")
    slower = int(input())
    print("Salary highest")
    supper = int(input())
    
    #backup
    jobs = ben_df.copy()

    #Filter based on the condition
    jobs = jobs[
          (jobs['template'] == template) &
          (jobs['type'] == wtype) &
          (jobs['shift'] <= shift) &
          (jobs['city'] == city) &
          (jobs['salary1'] >= slower) & 
          (jobs['salary2'] <= supper)
    ]

    #Only consider movies that have higher than m votes. Save this in a new dataframe q_jobs
    q_jobs = jobs.copy().loc[jobs['views'] >= 10]

    #     #Calculate score using the IMDB formula
    #     q_jobs['score'] = q_jobs.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average'])
    #     + (m/(m+x['vote_count']) * C)
    #     ,axis=1)

    #Sort jobs in descending order
    q_jobs = q_jobs.sort_values('views', ascending=False)

    return q_jobs

In [37]:
#Generate
result = build_chart(ben_df)
result

Input preferred template
Krankenpfleger
Working type:
teilzeit
Working shift
2
Where you want to work
Dresden
Salary lowest
2000
Salary highest
3000


,template,type,shift,city,zip,salary1,salary2,benefits,language,vertrag,futher,startDate,views
17425,Krankenpfleger,teilzeit,2,Dresden,01067-01465,2400,2600,"[Kostenübernahme Kinderbetreuung, Übernahme Fi...",0,1,[],1,508
16410,Krankenpfleger,teilzeit,2,Dresden,01067-01465,2400,2800,"[Mitarbeiterfrühstück, Kantine, Begrüßungsprämie]",1,2,[geförderte Weiterbildungsmöglichkeiten],0,161


In [36]:
result.shape[0]

2